In [14]:
import pandas as pd
import numpy as np

# Number 1: Anderson 9.3 #

Consider the case of a 1 MeV neutron incident on a nucleus of mass number $A = 100$. Using $R=R_0 A^{1/3}=(1.1)10^{-15} A^{1/3}$m, calculate the time required for a single transversal of the nuclear diameter by the neutron. Assume a square well nuclear potential with depth of 10 MeV. Compare this time with an estimate of the compound nucleus lifetime from the uncertainty principle for a resonance with level width $\Delta E=0.1$eV

In [28]:
R = 1.1 * 1e-15 * 100.**(1./3.)
R

5.105747716974057e-15

Using the classical relationship $T = \frac{1}{2} m_n v^2 \rightarrow v = \sqrt{2T / m_n}$

In [30]:
def beta_2(T, m):
    numerator = T * (T + 2 * m)
    denominator = (T + m) ** 2
    return numerator / denominator

In [32]:
beta = beta_2(11., 939.57) ** 0.5
beta

0.15169079422109602

To calculate the time interval, use the Heisenberg relationship $\Delta E \times \Delta t \geq h / 2 $

In [34]:
planck = 4.135e-22 # MeV * s

In [35]:
t = planck / 2. / 11.
t

1.8795454545454545e-23

This is the estimated compound nucleus lifetime (in seconds). Now we compare this to the transit time for the nuclear diameter

In [36]:
d = 2. * R

c = 3e8 # m/s
v = beta * c

t_transit = d / v

In [37]:
t_transit

2.2439277405026986e-22

# Number 2 #

Look up the thermal neutron cross sections for each stable isotope of Cd, and then using those
values determine the thermal neutron cross section for natural Cd (please include the natural
abundances of each isotope that you use in this calculation)

List of stable isotopes obtained from [ENDF](https://www.nndc.bnl.gov/sigma/index.jsp?as=116&lib=endfb7.1&nsub=10)

Natural abundance fractions obtained from [NC State](https://www.ncsu.edu/chemistry/msf/pdf/IsotopicMass_NaturalAbundance.pdf)

In [2]:
cadmium = pd.DataFrame(
    {
        "Isotope": [106, 108, 110, 111, 112, 113, 114, 116],
        "Abundance": [1.25, 0.89, 12.49, 12.80, 24.13, 12.22, 28.73, 7.49],
        "Cross Section": [1., 1.1, 11, 24, 2.2, 20600, 0.34, 0.075]
    }
)

In [5]:
cadmium["Fractional Cross Section Contribution"] = cadmium["Abundance"] * 1e-2 * cadmium["Cross Section"]

In [6]:
cadmium

Abundance  Cross Section  Isotope  Fractional Cross Section Contribution
0       1.25          1.000      106                               0.012500
1       0.89          1.100      108                               0.009790
2      12.49         11.000      110                               1.373900
3      12.80         24.000      111                               3.072000
4      24.13          2.200      112                               0.530860
5      12.22      20600.000      113                            2517.320000
6      28.73          0.340      114                               0.097682
7       7.49          0.075      116                               0.005618

In [7]:
sum(cadmium["Fractional Cross Section Contribution"])

2522.4223495000001

# Problem 3 #

Anderson 9.9, change the material to natural Cd. Use the cross section you determined in #3. 

What thickness of Cd will remove 95% of a beam of 100 eV neutrons?

Convert to macroscopic cross sections, using the A-number of each isotope as an approximation for the isotopic mass

In [8]:
def macroscopic_cross_section(A, microscopic):
    na = 6.022e23
    density = 8.65 # g/cm^3, from wikipedia for solid Cd
    
    macroscopic = microscopic * na * density / A
    
    return macroscopic

In [9]:
cadmium["Macroscopic Cross Section"] = macroscopic_cross_section(cadmium["Isotope"], cadmium["Cross Section"])

In [11]:
cadmium_mass_average = sum(cadmium["Abundance"] * 1e-2 * cadmium["Isotope"])
cadmium_mass_average

112.50800000000001

In [12]:
cadmium["Number Fraction"] = 6.022e23 * 8.65 / cadmium_mass_average * cadmium["Abundance"] * 1e-2

In [13]:
cadmium

Abundance  Cross Section  Isotope  Fractional Cross Section Contribution  \
0       1.25          1.000      106                               0.012500   
1       0.89          1.100      108                               0.009790   
2      12.49         11.000      110                               1.373900   
3      12.80         24.000      111                               3.072000   
4      24.13          2.200      112                               0.530860   
5      12.22      20600.000      113                            2517.320000   
6      28.73          0.340      114                               0.097682   
7       7.49          0.075      116                               0.005618   

   Macroscopic Cross Section  Number Fraction  
0               4.914179e+22     5.787400e+20  
1               5.305494e+22     4.120628e+20  
2               5.209030e+23     5.782770e+21  
3               1.126277e+24     5.926297e+21  
4               1.023202e+23     1.117200e+22  
5               9.496108e+26     5.657762e+21  
6               1.553570e+22     1.330176e+22  
7               3.367907e+21     3.467810e+21

$$ \phi = \phi_0 \exp{\left(-\Sigma_i * t\right)} $$
$$ t = \frac{-\log \left(\phi / \phi_0 \right) }{\Sigma} $$

In [20]:
Sigma = sum(cadmium['Number Fraction'] * cadmium['Cross Section'] * 1e-24)

In [24]:
t = -np.log(0.05) / Sigma

In [25]:
t

0.025651439296405364

To reduce the beam by 95%, you must have $2.57\times10^{-2}$ cm of Cd

# Problem 4 #

What is the relative probability of production of I-131 with respect to production of Cs-137 in thermal neutron fission of U-235? Use the double-hump curve on Figure 9.18. What is the relative probability of production of Mo-99 with respect to production of Cs-137?

From [Wikipedia](https://en.wikipedia.org/wiki/Fission_product_yield#Ordered_by_yield_.28thermal_neutron_fission_of_U-235.29), Cs-137 has a yield of 6.0899% while I-131 has a yield of 2.8336%. Mo-99 has a yield of 6.1%. Mo-99 is therefore about as likely to be produced as Cs-137, while I-131 is about half (46.6%) as likely to be produced as Cs-137.

In [23]:
2.8366 / 6.0899

0.46578761556018977